In [93]:
import warnings
warnings.filterwarnings('ignore')

In [94]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [95]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.metrics import classification_report_imbalanced

In [96]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [97]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [98]:
from sklearn import preprocessing

for col in df.columns:
    if df[col].dtype==type(object):
        encoder=preprocessing.LabelEncoder()
        df[col]=encoder.fit_transform(df[col])

x_cols=[i for i in df.columns if i not in ("loan_status")]
x=df[x_cols]
y=df['loan_status'].values.reshape(-1,1)

x.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [99]:
print(df['loan_status'].value_counts())

1    68470
0      347
Name: loan_status, dtype: int64


In [104]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)

#print(x_train)
#print(x_test)
#print(y_train)
#print(y_test)

0         355.29
1         873.53
2         485.44
3         335.33
4         484.36
          ...   
68812    1019.42
68813    1086.93
68814     571.33
68815    3661.36
68816    1041.03
Name: total_pymnt_inv, Length: 68817, dtype: float64


In [79]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaler=scaler.fit(x_train)
x_train_scaled=x_scaler.transform(x_train)
x_test_scaled=x_scaler.transform(x_test)

In [80]:
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model=BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model=rf_model.fit(x_train_scaled,y_train)

In [81]:
predictions=rf_model.predict(x_test_scaled)

In [82]:
acc_score=balanced_accuracy_score(y_test,predictions)
print(acc_score)

0.7337123387268567


In [83]:
cm = confusion_matrix(y_test,predictions)
print(cm)

[[   63    38]
 [ 2674 14430]]


In [84]:
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.62      0.84      0.04      0.73      0.51       101
          1       1.00      0.84      0.62      0.91      0.73      0.54     17104

avg / total       0.99      0.84      0.63      0.91      0.73      0.54     17205



In [85]:
# Print important features

tmp_frame=pd.DataFrame(columns=['feature','feature_importance'])
tmp_cols=df.columns.values
for x,i in enumerate(rf_model.feature_importances_):
    tmp_frame=tmp_frame.append({'feature':tmp_cols[x],'feature_importance':i},ignore_index=True)    

tmp_frame=tmp_frame.set_index('feature_importance')
tmp_frame=tmp_frame.sort_values(by='feature_importance',ascending=False)

#print(tmp_frame)

for (index_label, row_series) in tmp_frame.iterrows():
   print(row_series.values[0]+"    "+str(index_label))

total_pymnt_inv    0.07646711017201446
collection_recovery_fee    0.0681731200022894
out_prncp_inv    0.05699252404641979
total_pymnt    0.05424778840861491
total_rec_prncp    0.04735017423439608
int_rate    0.031245651401774252
issue_d    0.022619745828795626
mths_since_recent_bc    0.02244519942392171
installment    0.020409215004023765
tax_liens    0.01808337089191728
bc_open_to_buy    0.017004195413593538
pymnt_plan    0.01640559494238341
open_il_24m    0.016118934293676203
total_bal_il    0.01581716497633785
open_rv_24m    0.015630310867867408
initial_list_status    0.015494921031886601
out_prncp    0.015281199871082264
delinq_amnt    0.01470961095035287
mo_sin_old_il_acct    0.014693851231416522
tot_hi_cred_lim    0.01425370693271944
total_bal_ex_mort    0.014094010852862895
pub_rec    0.013976327240119397
acc_open_past_24mths    0.013793435235420952
tot_coll_amt    0.013416065136567646
mths_since_rcnt_il    0.0133985913791833
total_bc_limit    0.012881961311005559
annual_inc    

In [86]:
print(df.columns)

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [87]:
from imblearn.ensemble import EasyEnsembleClassifier

eec=EasyEnsembleClassifier(random_state=1,n_estimators=100)
eec=eec.fit(x_train_scaled,y_train)

In [88]:
eec_pred=eec.predict(x_test_scaled)

In [89]:
eec_accuracy=balanced_accuracy_score(y_test,eec_pred)
print(eec_accuracy)

0.9316600714093861


In [90]:
eec_cm=confusion_matrix(y_test,eec_pred)
print(eec_cm)

[[   93     8]
 [  983 16121]]


In [91]:
print(classification_report_imbalanced(y_test, eec_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.92      0.94      0.16      0.93      0.87       101
          1       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

